In [10]:
# import the random module
import random
import numpy as np
import matplotlib.pyplot as plt
from citipy import citipy
from datetime import datetime
import requests
import time
import pandas as pd

In [2]:
# Create a set of random latitude and longitude combinations.
lats = np.random.uniform(low=-90.000, high=90.000, size=1500)
lngs = np.random.uniform(low=-180.000, high=180.000, size=1500)
lat_lngs = zip(lats, lngs)
lat_lngs

In [3]:
# add the lat and longs to a list
coordinates = list(lat_lngs)

In [4]:
# create a list for holding the cities
cities = []

# identify the nearest city for each lat and long combo
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name
    
    # if the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)

#print the city count to confirm sufficient count
len(cities)

615

In [5]:
# Starting URL for Weather Map API Call.
# import the API key
from config import weather_api_key
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key

In [6]:
# Create an empty list to hold the weather data.
city_data = []
# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters.
record_count = 1
set_count = 1

Beginning Data Retrieval     
-----------------------------


In [7]:
# Loop through all the cities in the list.
for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
        time.sleep(60)

    # Create endpoint URL with each city.
    city_url = url + "&q=" + city.replace(" ","+")

    # Log the URL, record, and set numbers and the city.
    #print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # Add 1 to the record count.
    record_count += 1
    
    # Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        # Convert the date to ISO standard.
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Country": city_country,
                          "Date": city_date})

# If an error is experienced, skip the city.
    except:
        #print("City not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Processing Record 1 of Set 1 | zimapan
Processing Record 2 of Set 1 | albany
Processing Record 3 of Set 1 | moerai
Processing Record 4 of Set 1 | punta arenas
Processing Record 5 of Set 1 | bredasdorp
Processing Record 6 of Set 1 | castro
Processing Record 7 of Set 1 | victoria
Processing Record 8 of Set 1 | barrow
Processing Record 9 of Set 1 | kamenskoye
City not found. Skipping...
Processing Record 10 of Set 1 | ushuaia
Processing Record 11 of Set 1 | vaini
Processing Record 12 of Set 1 | rikitea
Processing Record 13 of Set 1 | obluche
City not found. Skipping...
Processing Record 14 of Set 1 | viedma
Processing Record 15 of Set 1 | bandarbeyla
Processing Record 16 of Set 1 | mataura
Processing Record 17 of Set 1 | codrington
Processing Record 18 of Set 1 | zachagansk
City not found. Skipping...
Processing Record 19 of Set 1 | yar-sale
Processing Record 20 of Set 1 | butaritari
Processing Record 21 of Set 1 | jamestown
Processing Record 22 of Set 1 | eureka
Processing Record 23 of S

In [8]:
len(city_data)

559

In [12]:
# Convert the array of dictionaries to a Pandas DataFrame.
city_data_df = pd.DataFrame(city_data)
new_column_order = ['City', 'Country', 'Date', 'Lat', 'Lng', 'Max Temp', 'Humidity', 'Cloudiness', 'Wind Speed']
city_data_df = city_data_df[new_column_order]

city_data_df.head(10)

,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,Zimapan,MX,2021-12-04 00:17:21,20.7500,-99.3500,56.79,56,72,5.44
1,Albany,US,2021-12-04 00:16:04,42.6001,-73.9662,31.80,60,1,8.19
2,Moerai,PF,2021-12-04 00:17:21,-22.4333,-151.3333,77.95,78,58,14.05
3,Punta Arenas,CL,2021-12-04 00:14:42,-53.1500,-70.9167,53.71,54,75,19.57
4,Bredasdorp,ZA,2021-12-04 00:17:21,-34.5322,20.0403,59.29,71,94,6.71
5,Castro,BR,2021-12-04 00:17:22,-24.7911,-50.0119,60.17,96,32,8.55
6,Victoria,HK,2021-12-04 00:09:56,22.2855,114.1577,64.26,39,12,1.01
7,Barrow,US,2021-12-04 00:14:40,71.2906,-156.7887,-12.98,70,1,18.41
8,Ushuaia,AR,2021-12-04 00:17:22,-54.8000,-68.3000,55.06,44,40,18.41
9,Vaini,TO,2021-12-04 00:17:23,-21.2000,-175.2000,84.36,84,40,14.97


In [14]:
# Create the output file (CSV).
output_data_file = "weather_data/cities.csv"
# Export the City_Data into a CSV.
city_data_df.to_csv(output_data_file, index_label="City_ID")